# 1. Đọc dữ liệu từ CSV

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("Fecom_Inc_Analysis") \
        .getOrCreate()

# load các file
order_df = spark.read.format("csv").options(header="True", delimiter=";", inferSchema="True").load("Orders.csv")
product_df = spark.read.format("csv").options(header="True", delimiter=";", inferSchema="True").load("Products.csv")
order_item_df = spark.read.format("csv").options(header="True", delimiter=";", inferSchema="True").load("Order_Items.csv")
customer_df = spark.read.format("csv").options(header="True", delimiter=";", inferSchema="True").load("Customer_List.csv")
order_review_df = spark.read.format("csv").options(header="True", delimiter=";", inferSchema="True").load("Order_Reviews.csv")



In [ ]:
order_df.printSchema()
product_df.printSchema()
order_item_df.printSchema()
customer_df.printSchema()
order_review_df.printSchema()

root
 |-- Order_ID: string (nullable = true)
 |-- Customer_Trx_ID: string (nullable = true)
 |-- Order_Status: string (nullable = true)
 |-- Order_Purchase_Timestamp: timestamp (nullable = true)
 |-- Order_Approved_At: timestamp (nullable = true)
 |-- Order_Delivered_Carrier_Date: timestamp (nullable = true)
 |-- Order_Delivered_Customer_Date: timestamp (nullable = true)
 |-- Order_Estimated_Delivery_Date: timestamp (nullable = true)

root
 |-- Product_ID: string (nullable = true)
 |-- Product_Category_Name: string (nullable = true)
 |-- Product_Weight_Gr: integer (nullable = true)
 |-- Product_Length_Cm: integer (nullable = true)
 |-- Product_Height_Cm: integer (nullable = true)
 |-- Product_Width_Cm: integer (nullable = true)

root
 |-- Order_ID: string (nullable = true)
 |-- Order_Item_ID: integer (nullable = true)
 |-- Product_ID: string (nullable = true)
 |-- Seller_ID: string (nullable = true)
 |-- Shipping_Limit_Date: timestamp (nullable = true)
 |-- Price: double (nullable = tr

In [ ]:
customer_df.show(20)

+--------------------+--------------------+--------------+----------------+--------------------+-------------+----------------+---------------------+---+------+
|     Customer_Trx_ID|       Subscriber_ID|Subscribe_Date|First_Order_Date|Customer_Postal_Code|Customer_City|Customer_Country|Customer_Country_Code|Age|Gender|
+--------------------+--------------------+--------------+----------------+--------------------+-------------+----------------+---------------------+---+------+
|1e959e1f5920cba43...|9765e039028279fd2...|    2023-07-08|      2023-07-09|            FR-75005|        Paris|          France|                   FR| 29|  Male|
|9877437582f263da7...|a75e134e7eb6f96e2...|    2024-03-23|      2024-04-11|           PL-00-001|       Warsaw|          Poland|                   PL| 38|  Male|
|fa6fbbb2080646aca...|2fdac27295500e820...|    2023-05-12|      2023-06-01|             NL-1012|    Amsterdam|     Netherlands|                   NL| 35|Female|
|a4c9ff14ae7620126...|e9ab8fd8ea96

# 2. Thống kê tổng số đơn hàng, số lượng khách hàng và người bán

In [ ]:
# Tổng số đơn hàng
total_order = order_df.count()
# tổng số khách hàng
total_cus = customer_df.select("Customer_Trx_ID").distinct().count()
# tổng số người bán
total_seller = order_item_df.select("seller_id").distinct().count()

print(f"Tổng số đơn hàng: {total_order}")
print(f"Tổng số khách hàng: {total_cus}")
print(f"Tổng số người bán: {total_seller}")

Tổng số đơn hàng: 99441
Tổng số khách hàng: 99442
Tổng số người bán: 3095


# 3. Phân tích số lượng đơn hàng theo quốc gia, sắp xếp theo thứ tự giảm dần.

In [ ]:
# customer_df.groupBy("Customer_Country").count().orderBy("count", ascending=False).show()

# join 2 bảng customer_df và order_df
customer_order_df = customer_df.join(order_df, customer_df.Customer_Trx_ID == order_df.Customer_Trx_ID, "inner")

# gop nhóm và sort
customer_order_df.groupBy("Customer_Country").count().orderBy("count", ascending=False).show()

+----------------+-----+
|Customer_Country|count|
+----------------+-----+
|         Germany|41754|
|          France|12848|
|     Netherlands|11629|
|         Belgium| 5464|
|         Austria| 5043|
|     Switzerland| 3640|
|  United Kingdom| 3382|
|          Poland| 2139|
|         Czechia| 2034|
|           Italy| 2025|
|           Spain| 1651|
|        Portugal| 1336|
|          Sweden|  975|
|         Denmark|  905|
|          Serbia|  746|
|          Norway|  716|
|        Slovakia|  534|
|        Slovenia|  495|
|          Turkey|  485|
|          Greece|  412|
+----------------+-----+
only showing top 20 rows


# 4. Phân tích số lượng đơn hàng nhóm theo năm, tháng đặt hàng (Hiển thị theo năm tăng dần, tháng giảm dần)


In [ ]:
from pyspark.sql.functions import year, month, desc

order_df.withColumn("order_year", year("Order_Purchase_Timestamp")) \
        .withColumn("order_month", month("Order_Purchase_Timestamp")) \
        .groupBy("order_year", "order_month") \
        .count() \
        .orderBy("order_year", desc("order_month")) \
        .show()

+----------+-----------+-----+
|order_year|order_month|count|
+----------+-----------+-----+
|      2022|         12|    1|
|      2022|         10|  324|
|      2022|          9|    4|
|      2023|         12| 5673|
|      2023|         11| 7544|
|      2023|         10| 4631|
|      2023|          9| 4285|
|      2023|          8| 4331|
|      2023|          7| 4026|
|      2023|          6| 3245|
|      2023|          5| 3700|
|      2023|          4| 2404|
|      2023|          3| 2682|
|      2023|          2| 1780|
|      2023|          1|  800|
|      2024|         10|    4|
|      2024|          9|   16|
|      2024|          8| 6512|
|      2024|          7| 6292|
|      2024|          6| 6167|
+----------+-----------+-----+
only showing top 20 rows


# 5. Thống kê điểm đánh giá trung bình, số lượng đánh giá theo từng mức

In [ ]:
order_review_df.show(10)

+--------------------+--------------------+------------+-----------------------+-------------------------+--------------------+-----------------------+
|           Review_ID|            Order_ID|Review_Score|Review_Comment_Title_En|Review_Comment_Message_En|Review_Creation_Date|Review_Answer_Timestamp|
+--------------------+--------------------+------------+-----------------------+-------------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                   NULL|                     NULL|    2024-01-18 00:00|    2024-01-18 21:46:00|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                   NULL|                     NULL|    2024-03-10 00:00|    2024-03-11 03:05:00|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                   NULL|                     NULL|    2024-02-17 00:00|    2024-02-18 14:36:00|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|                   NULL|     I r

In [ ]:
order_review_df.groupBy("Review_Score").count().show()

+----------------+-----+
|    Review_Score|count|
+----------------+-----+
|               3| 8179|
|            NULL|   45|
|               5|57328|
|2024-07-05 11:11|    1|
|               1|11424|
|               4|19141|
|               2| 3151|
|2024-04-07 04:19|    1|
+----------------+-----+



In [ ]:
  from pyspark.sql import functions as F
from pyspark.sql.functions import col

valid_scores = ['1', '2', '3', '4', '5']
order_review_cleaned = order_review_df.filter(col("Review_Score").isin(valid_scores))

In [ ]:
order_review_cleaned.printSchema()

root
 |-- Review_ID: string (nullable = true)
 |-- Order_ID: string (nullable = true)
 |-- Review_Score: string (nullable = true)
 |-- Review_Comment_Title_En: string (nullable = true)
 |-- Review_Comment_Message_En: string (nullable = true)
 |-- Review_Creation_Date: string (nullable = true)
 |-- Review_Answer_Timestamp: timestamp (nullable = true)



In [ ]:
# in tất cả giá trị trong cột review_score
order_review_cleaned.select("Review_Score").distinct().show()

+------------+
|Review_Score|
+------------+
|           3|
|           5|
|           1|
|           4|
|           2|
+------------+



In [ ]:
print("DataFrame sau khi làm sạch:")
order_review_cleaned.show(10)
print(f"Tổng số hàng còn lại sau khi làm sạch: {order_review_cleaned.count()}")

DataFrame sau khi làm sạch:
+--------------------+--------------------+------------+-----------------------+-------------------------+--------------------+-----------------------+
|           Review_ID|            Order_ID|Review_Score|Review_Comment_Title_En|Review_Comment_Message_En|Review_Creation_Date|Review_Answer_Timestamp|
+--------------------+--------------------+------------+-----------------------+-------------------------+--------------------+-----------------------+
|7bc2406110b926393...|73fc7af87114b3971...|           4|                   NULL|                     NULL|    2024-01-18 00:00|    2024-01-18 21:46:00|
|80e641a11e56f04c1...|a548910a1c6147796...|           5|                   NULL|                     NULL|    2024-03-10 00:00|    2024-03-11 03:05:00|
|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                   NULL|                     NULL|    2024-02-17 00:00|    2024-02-18 14:36:00|
|e64fb393e7b32834b...|658677c97b385a9be...|           5|    

In [ ]:
review_score_col = F.col("Review_Score").cast("int")
avg_agg_expr = F.avg(review_score_col)
order_review_cleaned.select(avg_agg_expr).show()

+------------------------------+
|avg(CAST(Review_Score AS INT))|
+------------------------------+
|            4.0864214950162765|
+------------------------------+



In [ ]:
# Nhóm dữ liệu theo Review_Score và đếm số lượng cho mỗi nhóm
# Sau đó sắp xếp kết quả theo Review_Score tăng dần
order_review_cleaned.groupBy("Review_Score").count().orderBy("Review_Score").show()

+------------+-----+
|Review_Score|count|
+------------+-----+
|           1|11424|
|           2| 3151|
|           3| 8179|
|           4|19141|
|           5|57328|
+------------+-----+



# 6. Tính doanh thu (giá sản phẩm + phí vận chuyển) trong năm 2024 và nhóm the danh mục sản phẩm

In [ ]:
order_item_df.show(10)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            Order_ID|Order_Item_ID|          Product_ID|           Seller_ID|Shipping_Limit_Date| Price|Freight_Value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2023-09-19 09:45:00|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2023-05-03 11:05:00| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2024-01-18 14:48:00| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2024-08-15 10:10:00| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2023-02-13 13:57:00| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [ ]:

# Tạo cột mới 'total_price' bằng cách cộng 'Price' và 'Freight_Value'
order_item_df = order_item_df.withColumn("total_value", F.col("Price") + F.col("Freight_Value"))

# nhóm 2 bảng
product_order_df = product_df.join(order_item_df, product_df.Product_ID == order_item_df.Product_ID, "inner")

# nhóm với bảng order
product_order_df = product_order_df.join(order_df, product_order_df.Order_ID == order_df.Order_ID, "inner")

product_order_df.show(10)



+--------------------+---------------------+-----------------+-----------------+-----------------+----------------+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+------------------+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|          Product_ID|Product_Category_Name|Product_Weight_Gr|Product_Length_Cm|Product_Height_Cm|Product_Width_Cm|            Order_ID|Order_Item_ID|          Product_ID|           Seller_ID|Shipping_Limit_Date| Price|Freight_Value|       total_value|            Order_ID|     Customer_Trx_ID|Order_Status|Order_Purchase_Timestamp|  Order_Approved_At|Order_Delivered_Carrier_Date|Order_Delivered_Customer_Date|Order_Estimated_Delivery_Date|
+--------------------+---------------------+-----------------+-----------------+-----------------+----------------+-

In [ ]:

df_2024 = (product_order_df.withColumn("order_ts",F.to_timestamp("Order_Purchase_Timestamp") ).filter(F.year("order_ts") == 2024))

revenue_by_category_2024 = (
    df_2024
    .groupBy("Product_Category_Name")
    .agg(
        F.sum("total_value").alias("total_revenue_2024")
    )
    .orderBy(F.desc("total_revenue_2024"))
)




In [ ]:
revenue_by_category_2024.show()

+---------------------+------------------+
|Product_Category_Name|total_revenue_2024|
+---------------------+------------------+
|        Health_Beauty|  885191.119999997|
|        Watches_Gifts|  771986.750000001|
|       Bed_Bath_Table|  650794.700000002|
|       Sports_Leisure| 621999.3399999994|
| Computers_Accesso...| 594771.0400000002|
|           Housewares| 491576.9600000012|
|      Furniture_Decor| 476466.1300000007|
|                 Auto| 404210.5700000002|
|                 Baby| 299052.5599999998|
|           Cool_Stuff| 273910.0500000001|
|         Garden_Tools|259068.31999999983|
|            Telephony| 217452.1299999995|
|            Perfumery|204562.53999999992|
|                 Toys|200634.07000000007|
|     Office_Furniture| 181745.7300000001|
|           Stationery|164743.84999999986|
|             Pet_Shop|152804.94000000012|
| Construction_Tool...|141187.33999999985|
|          Electronics| 134265.4499999999|
|  Musical_Instruments|121476.30999999997|
+----------

# 7. Xác định sản phẩm có số lượng bán ra cao nhất và tính điểm đánh giá trung bình cho từng sản phẩm

In [ ]:
from pyspark.sql.functions import col, count, avg, round, desc
full_data = order_item_df.join(order_review_cleaned, "Order_ID", "inner") \
    .join(order_df, "Order_ID", "inner")

result_df = full_data.groupBy("Product_ID") \
    .agg(
        count("Product_ID").alias("Total_Sold"),         # Đếm số lượng bán ra
        round(avg("Review_Score"), 2).alias("Avg_Score") # Tính điểm TB và làm tròn 2 số
    ) \
    .orderBy(desc("Total_Sold")) # Sắp xếp giảm dần theo số lượng bán




In [ ]:
result_df.show(10)

+--------------------+----------+---------+
|          Product_ID|Total_Sold|Avg_Score|
+--------------------+----------+---------+
|aca2eb7d00ea1a7b8...|       524|     4.02|
|422879e10f4668299...|       486|     3.95|
|99a4788cb24856965...|       482|      3.9|
|389d119b48cf3043d...|       391|     4.12|
|368c6c730842d7801...|       388|     3.92|
|53759a2ecddad2bb8...|       373|     3.87|
|d1c427060a0f73f6b...|       340|     4.19|
|53b36df67ebb7c415...|       320|     4.19|
|154e7e31ebfa09220...|       292|     4.32|
|3dd2a17168ec895c7...|       272|     4.21|
+--------------------+----------+---------+
only showing top 10 rows
